In [8]:
import pandas as pd
import numpy as np
from walkscore import WalkScoreAPI

In [9]:
# Replace with your own file path to csv of parcels
FilePath = '/Users/hoperuse/summer2021internship/State Surplus/vacant_transportation_authority.csv'
dot = pd.read_csv(FilePath)

In [10]:
# Add your own WalkScore API key: https://www.walkscore.com/professional/api.php
api_key = 'XXXXX'

# Using Python library to access WalkScore
walkscore_api = WalkScoreAPI(api_key = api_key)

# Creating three columns for the corresponding scores in the DOT frame, and filling them with initial Null values
dot['TransitScore'] = np.nan
dot['WalkScore'] = np.nan
dot['BikeScore'] = np.nan

for i in range(len(dot.index)):
    lat = dot.loc[i,'latitude']
    long = dot.loc[i,'longitude']
    
    # If a given tuple doesn't have valid longitude / latitude coordinates, the API cannot calculate any scores, so set all to nan and 
    # continue to the next iteration
    if np.isnan(lat) or np.isnan(long): 
        dot.at[i, 'TransitScore'] = np.nan
        dot.at[i,'WalkScore'] = np.nan
        dot.at[i, 'BikeScore'] = np.nan
        continue
    
    # Collecting the individual values for street number, street, and zipcode to construct an address string
    streetNum = dot.loc[i,'addr_num']
    street = dot.loc[i,'addr_str']
    zipcode = dot.loc[i,'addr_zip']

    # If any of the address values are null, the WalkScore API will be called just using the longitude and latitidue coordinates
    if np.isnan(streetNum) or np.isnan(street) or np.isnan(zipcode):
       result = walkscore_api.get_score(latitude = lat, longitude = long) 
    else:
        if len(str(zipcode)) == 4:
            zipcode = '0' + str(zipcode)
        address = str(streetNum) + ' ' + str(street) + ', MA ' + str(zipcode)
        result = walkscore_api.get_score(latitude = lat, longitude = long, address=address)  

    if np.isnan(result.transit_score):
        # The walkscore API does not offer transit score calculations for many cities, whereas it will always return a walk / bike score, 
        # so a lot of the returned transit values are empty
        dot.at[i, 'TransitScore'] = np.nan
    else:
        dot.at[i, 'TransitScore'] = result.transit_score
    dot.at[i,'WalkScore'] = result.walk_score
    dot.at[i, 'BikeScore'] = result.bike_score


In [11]:
# load dataframe with transit information to CSV
dot.to_csv('DOTtransitScores.csv')